<a href="https://colab.research.google.com/github/medha-sagar/CSE547-BigDataForMachineLearning/blob/master/CSE547_Q2_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Get Dependencies for Google 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Load Data
id='1rqNLakXn3uthZtXvHSzD6s2xn-w0NscJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('browsing.txt')

In [ ]:
#Get Dependencies for Spark
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#Get Dependencies for Spark
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter

%matplotlib inline

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [ ]:
browsing = sc.textFile("browsing.txt")
browsing_list = browsing.collect()

In [ ]:
items = browsing.flatMap(lambda browsing: browsing.split(" "))
items = items.filter(lambda item: item not in "")

In [ ]:
#Item Frequency
itemFreq = items.map(lambda item: (item, 1)).reduceByKey(lambda a,b:a +b)
itemFreqDict = itemFreq.collectAsMap()
itemFreqDict100 = {k:v for (k,v) in itemFreqDict.items() if v >= 100}

In [ ]:
print(len(itemFreq.collect()))
print(len(itemFreqDict100))

12592
647


In [ ]:
#Create item item_pair frequency
item_pair_100_combinations = set(combinations(itemFreqDict100.keys(), 2))

In [ ]:
temp = []
for item_pair in item_pair_100_combinations:
  temp.append(tuple(sorted(item_pair)))
item_pair_100C_dict =  Counter(temp)

In [ ]:
basket_pairs = {}
for basket in browsing_list:
  basket_item_list = sorted(basket.split(" ")[:-1])
  basket_combinations = combinations(basket_item_list,2)
  for basket_combination in basket_combinations:
    if tuple(basket_combination) in item_pair_100C_dict.keys():
      if tuple(sorted(basket_combination)) in basket_pairs.keys():
        basket_pairs[tuple(basket_combination)] +=1
      else:
        basket_pairs[tuple(basket_combination)] = 1

In [ ]:
basket_pairs = {k:v for (k,v) in basket_pairs.items() if v >= 100}

In [ ]:
list_pair = [list(pair) for pair in item_pair_100C_dict.keys()]

In [ ]:
common_items = []
for k, v in basket_pairs.items():
  if [k[0],k[1]] in list_pair:
    common_items.append(k)

In [ ]:
df = pd.DataFrame(
    [common_item[0], common_item[1], basket_pairs[tuple([common_item[0], common_item[1]])]/itemFreqDict100[common_item[0]]] for common_item in common_items
)

df2 = pd.DataFrame(
    [common_item[0], common_item[1], basket_pairs[tuple([common_item[0], common_item[1]])]/itemFreqDict100[common_item[1]]] for common_item in common_items
)

res = df.append(df2)

In [ ]:
res.columns = ["itemA", "itemB", "Conf"]
print(res.sort_values("Conf",ascending=False).head(5))

        itemA     itemB      Conf
624  DAI93865  FRO40251  1.000000
134  FRO40251  GRO85051  0.999176
130  FRO40251  GRO38636  0.990654
591  ELE12951  FRO40251  0.990566
208  DAI88079  FRO40251  0.986726


In [ ]:
#Create item item_pair frequency
item_tri_100_combinations = set(combinations(itemFreqDict100.keys(), 3))
temp1 = []
for item_tri in item_tri_100_combinations:
  temp1.append(tuple(sorted(list(item_tri))))
item_tri_100C_dict =  Counter(temp1)
len(item_tri_100C_dict)

44930915

In [ ]:
len(item_tri_100C_dict)

44930915

In [ ]:
#item_tri_100C_dict = {k:v for (k,v) in item_tri_100C_dict.items() if v >= 100}

In [ ]:
print(temp1[1:2])

[('FRO61250', 'FRO71487', 'SNA38046')]


In [ ]:
basket_three_items = {}
for basket in browsing_list:
  basket_item_list = sorted(basket.split(" ")[:-1])
  basket_combinations = combinations(basket_item_list,3)
  for basket_combination in basket_combinations:
    if tuple(basket_combination) in item_tri_100C_dict.keys():
      if tuple(sorted(basket_combination)) in basket_three_items.keys():
        basket_three_items[tuple(basket_combination)] +=1
      else:
        basket_three_items[tuple(basket_combination)] = 1  


In [ ]:
basket_triplets = basket_three_items

In [ ]:
basket_triplets = {k:v for (k,v) in basket_triplets.items() if v >= 100}

In [ ]:
len(basket_triplets)

233

In [ ]:
list_triplets = [list(triplet) for triplet in item_tri_100C_dict.keys()]

In [ ]:
common_triplets = []
for k, v in basket_triplets.items():
  if [k[0],k[1],k[2]] in list_triplets:
    common_triplets.append(k)

In [ ]:
df1 = df = pd.DataFrame(
    [common_triplet[0], common_triplet[1], common_triplet[2], basket_triplets[tuple([common_triplet[0], common_triplet[1], common_triplet[2]])]/basket_pairs[tuple([common_triplet[0], common_triplet[1]])]] for common_triplet in common_triplets
)

df2 = df = pd.DataFrame(
    [common_triplet[1], common_triplet[2], common_triplet[0], basket_triplets[tuple([common_triplet[0], common_triplet[1], common_triplet[2]])]/basket_pairs[tuple([common_triplet[1], common_triplet[2]])]] for common_triplet in common_triplets
)

df3 = df = pd.DataFrame(
    [common_triplet[0], common_triplet[2], common_triplet[1], basket_triplets[tuple([common_triplet[0], common_triplet[1], common_triplet[2]])]/basket_pairs[tuple([common_triplet[0], common_triplet[2]])]] for common_triplet in common_triplets
)


res2 = df1.append(df3)

In [ ]:
res2.columns = ["itemA", "itemB", "itemC", "Conf"]
print(res2.sort_values(["Conf", "itemA", "itemB", "itemC"], ascending=[False, True, True, True]).head(5))

        itemA     itemB     itemC  Conf
229  DAI23334  ELE92920  DAI62779   1.0
137  DAI31081  GRO85051  FRO40251   1.0
71   DAI55911  GRO85051  FRO40251   1.0
17   DAI62779  DAI88079  FRO40251   1.0
52   DAI75645  GRO85051  FRO40251   1.0
